Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *
from tickers import tickers

from scipy.stats import norm as norm
from yahoo_query import *

In [44]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

stock_list = pd.read_csv('optionablestocks.csv')['OPTION SYMBOL'].tolist()
stock_list = list(set(tickers[3:] + stock_list))

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
#     earnings_info['earnBeatsBefore'] = 0
#     earnings_info['earnMissBefore'] = 0

#     for idx, row in earnings_info.iterrows():
#         earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
#         earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
#     earnings_info = earnings_info.shift(1)


#     earnings_moves = past_earnings(ticker).sort_index()
#     earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
#                                     (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


#     earnings_df = pd.concat([earnings_info.reset_index(), 
#                              earnings_moves.reset_index()], axis = 1)
    earnings_info.columns = ['quarter' if col == 'index' else col for col in earnings_info.columns.tolist()]
    earnings_info['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_info, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    #earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [70]:
# earnings, ks, failed_list = download_yahoo_data(stock_list, retries = 10)
# earnings = earnings.drop(earnings.isnull().sum().sort_values().tail(13).index.tolist(), axis = 1)
# earnings.to_csv('full_data.csv')

earnings_pct = []
for name in earnings.Underlying.drop_duplicates().tolist():
    pct = earnings[earnings['Underlying'] == name].sort_index()
    pct[pct.columns.tolist()[1:]] = pct[pct.columns.tolist()[1:]].pct_change()
    earnings_pct.append(pct.tail(len(pct)-1))

earnings_pct = pd.concat(earnings_pct, axis = 0)

In [71]:
earnings_pct

,Underlying,accountsPayable,capitalExpenditures,capitalSurplus,cash,changeInCash,changeToAccountReceivables,changeToInventory,changeToLiabilities,changeToNetincome,...,totalCashFromOperatingActivities,totalCashflowsFromInvestingActivities,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,totalStockholderEquity,treasuryStock
2017-09-30,USAT,-0.114800,-0.188880,NaN,3.069831,-8.770606,4.320000,9.218487,-1.414136,-0.764398,...,-1.405897,-0.033673,1.043778,-0.090752,-0.087425,-0.249579,3.545918,-0.252909,0.643646,NaN
2017-12-31,USAT,0.673000,-0.218750,NaN,-0.703374,-1.932498,-0.342732,-0.404057,-5.205720,54.450000,...,2.117413,68.529039,-0.335661,0.307384,1.420115,0.273192,3.041526,0.268923,0.078420,NaN
2018-03-31,USAT,0.238528,0.597419,NaN,0.111855,-1.047171,2.661582,-1.091076,-0.247861,-1.152590,...,0.452601,-0.982641,0.151631,0.165182,0.070955,0.069154,-0.382116,0.102320,0.015680,NaN
2017-12-31,RARE,-0.043282,-0.129719,0.087111,0.663516,-2.320887,NaN,NaN,5.736459,0.542136,...,0.319875,0.917014,-0.310358,0.105087,0.544115,0.111273,-2.584410,11.191919,0.048955,-0.232847
2018-03-31,RARE,-0.148211,-0.471635,0.249811,1.256289,2.147676,NaN,NaN,-0.920977,-6.029421,...,0.093556,-2.243836,1.157332,-0.189132,-0.126805,0.149236,-18.821057,3.422949,0.887207,-0.833451
2018-06-30,RARE,0.362663,1.898947,0.021448,-0.511075,-1.918462,NaN,NaN,-3.044799,-0.807770,...,-0.149757,-0.266923,0.039828,0.147023,0.089206,0.004946,-0.666554,0.198277,-0.027203,-0.308668
2017-12-31,PHH,-0.048780,0.000000,0.012545,0.097166,-1.094675,NaN,NaN,NaN,-21.052632,...,-6.773585,-0.435644,-0.217756,-0.791420,-0.255562,-0.273333,-0.877551,-0.107438,-0.069024,-0.100000
2018-03-31,PHH,1.000000,0.000000,0.000000,-0.114391,-1.979167,NaN,NaN,NaN,-0.419948,...,-0.467320,-0.912281,-0.255368,0.070922,-0.200323,-0.348624,-2.666667,-0.675926,-0.054250,0.000000
2018-06-30,PHH,-0.141026,0.000000,0.001770,-0.056250,-0.127660,NaN,NaN,NaN,-1.013575,...,-1.337423,-0.600000,-0.057673,-0.099338,-0.037374,-0.042254,-1.300000,0.000000,-0.065010,0.000000
2017-12-31,FIT,0.191931,0.684311,0.026996,0.218344,-2.611618,1.776470,-7.222277,-0.575152,-0.567126,...,9.113479,-1.103919,0.123077,0.241274,0.216986,0.324162,72.335749,0.454074,-0.009225,-0.999300


In [ ]:
ks = pd.read_csv('keyStats.csv',index_col = 0).drop_duplicates()
earnings = pd.read_csv('earningsHist_unshifted.csv', index_col = 0).drop_duplicates()

In [ ]:
ks = ks.drop(ks.T.isnull().sum(axis = 1).sort_values(ascending = False).head(18).index.tolist(),
             axis = 1)

score_cols = ['52WeekChange', 'earningsGrowth',
              'earningsQuarterlyGrowth', 'forwardPE',
              'grossMargins', 'grossProfits',
              'pegRatio', 'profitMargins', 
              'returnOnAssets',
              'returnOnEquity', 'revenueGrowth', 'sector']




earnings.quarter = pd.to_datetime(earnings.quarter)
earnings = earnings.groupby(['Underlying','quarter']).sum().reset_index()
ticker = 'A'
latest_earnings = []
for ticker in earnings.Underlying.drop_duplicates().tolist():
    latest_earnings.append(earnings[earnings['Underlying'] == ticker][earnings.quarter == max(earnings[earnings['Underlying'] == ticker].quarter)])
latest_earnings = pd.concat(latest_earnings, axis = 0)
latest_earnings.index = latest_earnings.Underlying

earn_cols = ['quarter', '1Year', '1month', '3month', '6month','totalLiab',
             'totalStockholderEquity','netIncome',
             'operatingIncome',
             'totalRevenue','dividendsPaid',
             'investments',
             'longTermInvestments',
             'shortTermInvestments']

latest_earnings = latest_earnings[earn_cols]


ks = ks[score_cols]

names = ks.join(latest_earnings).drop_duplicates()

names['dividendsPaid'] = -names['dividendsPaid']

names['payoutRatio'] = names.dividendsPaid/names.netIncome
names['debtEquityRatio'] = names.totalLiab/names.totalStockholderEquity
names['netMargin'] = names.netIncome/names.totalRevenue
names['roic'] = names.operatingIncome/(names.longTermInvestments + names.shortTermInvestments)

names = names[['debtEquityRatio',
       '52WeekChange','earningsGrowth',
       'earningsQuarterlyGrowth', 'forwardPE',
       'grossMargins', 'grossProfits',
       'pegRatio', 'profitMargins', 
       'returnOnAssets',
       'returnOnEquity', 'revenueGrowth', 'sector']]

names = names[names['52WeekChange'] != 'Infinity']
names['52WeekChange'] = pd.to_numeric(names['52WeekChange'])

removes = pd.DataFrame(names.isnull().sum(axis = 1).sort_values(ascending = False))
removes = removes[removes[0] <= 3]

names = names[names.index.isin(removes.index.tolist())].fillna(0)

names_mean = names.groupby('sector').mean()
names_mean = names[['sector']].join(names_mean, on = 'sector')

names_high = names.groupby('sector').max()
names_high = names[['sector']].join(names_high, on = 'sector')

names_low = names.groupby('sector').min()
names_low = names[['sector']].join(names_low, on = 'sector')

names_std = names.groupby('sector').std()
names_std = names[['sector']].join(names_std, on = 'sector')

del names['sector'], names_mean['sector'], names_high['sector'], names_low['sector'], names_std['sector']

score = (names - names_mean)/names_std#(names_high - names_low)

del score['forwardPE']

score['score'] = ((score['52WeekChange'])/3 -
                   (score['debtEquityRatio'] + score['pegRatio'])/3 +
                   (score['earningsGrowth'] + score['revenueGrowth'])/3)
score.join(ks[['sector']]).sort_values('score',ascending = False).to_csv('leaps_scores.csv')